`05-rag-conversation.ipynb`

In [ ]:
%pip install -q langchain_pinecone

In [29]:
from dotenv import load_dotenv
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_pinecone import PineconeVectorStore
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 대화 기록 남기기용
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# 우리가 만든 함수
from pinecone_utils import create_embeddings, get_vectorstore


load_dotenv()

True

In [7]:
loader = PyMuPDFLoader('./data.pdf')
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
split_docs = text_splitter.split_documents(docs)

index_name = 'gaicc-1'
# 최초 1회만 pdf 를 Pinecone Vectorstore에 저장.
# create_embeddings(index_name=index_name, docs=split_docs)


In [18]:
vectorstore.similarity_search('구글 관련 정보를 모두 알려줘')

[Document(metadata={'author': 'dj', 'creationDate': "D:20231208132838+09'00'", 'creator': 'Hwp 2018 10.0.0.13462', 'file_path': './data.pdf', 'format': 'PDF 1.4', 'keywords': '', 'modDate': "D:20231208132838+09'00'", 'page': 13.0, 'producer': 'Hancom PDF 1.3.0.542', 'source': './data.pdf', 'subject': '', 'title': '', 'total_pages': 23.0, 'trapped': ''}, page_content='1. 정책/법제  \n2. 기업/산업 \n3. 기술/연구 \n 4. 인력/교육\n구글, 앤스로픽에 20억 달러 투자로 생성 AI 협력 강화 \nn 구글이 앤스로픽에 최대 20억 달러 투자에 합의하고 5억 달러를 우선 투자했으며, 앤스로픽은 \n구글과 클라우드 서비스 사용 계약도 체결\nn 3대 클라우드 사업자인 구글, 마이크로소프트, 아마존은 차세대 AI 모델의 대표 기업인 \n앤스로픽 및 오픈AI와 협력을 확대하는 추세\nKEY Contents\n£ 구글, 앤스로픽에 최대 20억 달러 투자 합의 및 클라우드 서비스 제공\nn 구글이 2023년 10월 27일 앤스로픽에 최대 20억 달러를 투자하기로 합의했으며, 이 중 5억 \n달러를 우선 투자하고 향후 15억 달러를 추가로 투자할 방침\n∙구글은 2023년 2월 앤스로픽에 이미 5억 5,000만 달러를 투자한 바 있으며, 아마존도 지난 9월 \n앤스로픽에 최대 40억 달러의 투자 계획을 공개'),
 Document(metadata={'author': 'dj', 'creationDate': "D:20231208132838+09'00'", 'creator': 'Hwp 2018 10.0.0.13462', 'file_path': './data.pdf', 'format

In [41]:
vectorstore = get_vectorstore(index_name)
retriever = vectorstore.as_retriever()
text = '''
넌 질문-답변을 도와주는 AI 비서야.
먼저 Previous Chat History 에 이전 대화 내용이 들어가 있을 거야.
해당 내용을 잘 읽고, 아래 제공되는 Context를 통해서 사용자 Question에 대해 답을 해줘야해.


Context에는 직접적으로 없어도, 추론하거나 계산할 수 있는 답변은 최대한 만들어 봐.
그리고 관련 내용을 말할때는, pdf 몇 페이지이서 찾은 내용인지도 말해줘.

답은 적절히 줄바꿈을 통해 문단을 나눠줘. 답변은 한국어로 만들어 줘. 

# Previous Chat History
{chat_history}

# Question:
{question}

# Context:
{context}

# Answer:
'''
prompt = PromptTemplate.from_template(text)
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.5)
parser = StrOutputParser()


from operator import itemgetter
chain = (
    {   
        'chat_history': itemgetter('chat_history'),
        'context': itemgetter('question') |  retriever,
        'question': itemgetter('question')  
    }
    | prompt
    | llm
    | parser
)



In [32]:
# 기존 대화 내용 기록 보관소 dict
store = {}

def get_session_history(session_id):
    print(f'[대화ID]: {session_id}를 불러옵니다...')
    # 채팅 보관소에 없다면
    if session_id not in store:
        # 새로 만든다
        store[session_id] = ChatMessageHistory()
    
    # 있으면 그대로 주고, 없으면 방금 위에서 만든걸 준다.
    return store[session_id]

In [34]:
rag_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key='question',
    history_messages_key='chat_history'
)

In [40]:
rag_with_history.invoke(
    # 우리가 실제로 물어볼 질문
    {'question': '삼성전자 관련 다 말해줘'},
    # 이 대화가 어떤 대화기록 id (session_id)에 소속되는지 작성되었는지
    config={'configurable': {'session_id': 'qwer'}}
)

[대화ID]: qwer를 불러옵니다...


'삼성전자가 삼성 가우스라는 온디바이스 AI 모델을 공개했고, 이 모델은 언어, 코드, 이미지의 3가지 모델로 구성되어 안전한 데이터를 활용하여 개인정보 침해를 방지하는 장점을 가지고 있습니다. 또한, 삼성전자는 이 모델을 다양한 제품에 탑재할 계획이라고 합니다. 이는 온디바이스 AI 기술이 주요 기술 트렌드로 인식되고 있으며, 삼성 스마트폰이 메타의 라마2나 구글 어시스턴트와 경쟁할 것으로 예상되는 내용입니다. (참고: data.pdf, 12페이지, 22페이지)'

In [38]:
store

{'qwer': InMemoryChatMessageHistory(messages=[HumanMessage(content='삼성전자 관련 정보를 다 가져와'), AIMessage(content='삼성전자는 온디바이스에서 작동 가능한 삼성 가우스라는 자체 개발 생성 AI 모델을 공개했습니다. 삼성 가우스는 언어, 코드, 이미지의 3개 모델로 구성되어 있으며, 안전한 데이터를 통해 학습되어 개인정보를 침해하지 않는 장점을 가지고 있습니다. 또한, 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획이라고 합니다. 이 모델은 언어 모델, 코드 모델, 이미지 모델로 구성되어 있으며, 각각의 모델은 다양한 기능을 제공합니다. 또한, 온디바이스 AI가 주요 기술 트렌드로 인식되고 있고, 삼성 스마트폰이 가우스를 탑재하여 메타의 라마2나 구글 어시스턴트를 탑재한 기기와 경쟁할 것으로 예상되고 있습니다. (참고: data.pdf, 12페이지, 22페이지)'), HumanMessage(content='위에 답변과 비슷한 소식들은 없어?'), AIMessage(content='예, 최근 소식 중에서는 삼성전자가 삼성 가우스라는 온디바이스 AI 모델을 공개한 것이 큰 소식입니다. 이 모델은 언어, 코드, 이미지의 3가지 모델로 구성되어 있으며, 안전한 데이터를 활용하여 학습되어 개인정보 침해를 방지하는 장점을 가지고 있습니다. 또한, 삼성전자는 이 모델을 다양한 제품에 탑재할 계획이라고 합니다. 이는 온디바이스 AI 기술이 주요 기술 트렌드로 인식되고 있고, 삼성 스마트폰이 메타의 라마2나 구글 어시스턴트와 경쟁할 것으로 예상되는 중요한 소식입니다. (참고: data.pdf, 12페이지, 22페이지)')]),
 'asdf': InMemoryChatMessageHistory(messages=[HumanMessage(content='구글정보 다 가져와'), AIMessage(content='구글은 2023년 2월 앤스로픽에 이미 5억 5,000만 달러를 투자한 바 있으며, 아